<a href="https://colab.research.google.com/github/isabelavs/alura-imersao-dados/blob/main/5_Valida%C3%A7%C3%A3o_de_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 5: Validação de modelo e Overfit

In [1]:
import pandas as pd

fonte = "https://github.com/alura-cursos/imersao-dados-2-2020/blob/master/MICRODADOS_ENEM_2019_SAMPLE_43278.csv?raw=true"

dados = pd.read_csv(fonte)

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
provas = ["NU_NOTA_REDACAO", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_MT", "NU_NOTA_LC"]

dados["NU_NOTA_TOTAL"] = dados[provas].sum(axis = 1)

dados_sem_notas_zero = dados.query("NU_NOTA_TOTAL != 0")

In [5]:
provas_entrada = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
prova_saida = "NU_NOTA_MT"

dados_sem_notas_zero = dados_sem_notas_zero[provas].dropna()

notas_entrada = dados_sem_notas_zero[provas_entrada]
notas_saida = dados_sem_notas_zero[prova_saida]

In [6]:
x = notas_entrada
y = notas_saida

from sklearn.model_selection import train_test_split

SEED = 4321

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25,
                                                        random_state=SEED)

In [9]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

modelo = LinearSVR(random_state=SEED)
modelo.fit(x_treino, y_treino)
predicoes_mt = modelo.predict(x_teste)
mean_squared_error(y_teste, predicoes_mt)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


6604.415306219566

In [15]:
from sklearn.tree import DecisionTreeRegressor

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25)
modelo_arvore = DecisionTreeRegressor(max_depth = 3)
modelo_arvore.fit(x_treino, y_treino)
predicoes_mt_arvore = modelo_arvore.predict(x_teste)
mean_squared_error(y_teste, predicoes_mt_arvore)

5971.240593640753

In [21]:
from sklearn.model_selection import cross_validate

modelo_arvore = DecisionTreeRegressor(max_depth=2)
resultado = cross_validate(modelo_arvore, x, y, scoring="neg_mean_squared_error")
resultado["test_score"]*-1

array([6317.91361711, 6390.69330384, 6519.97057993, 6702.12730215,
       6851.82019408])

In [23]:
media = (resultado["test_score"]*-1).mean()
media

6556.50499941891

In [24]:
resultado = cross_validate(modelo_arvore, x, y, cv=10, scoring="neg_mean_squared_error")
resultado["test_score"]*-1

array([6118.6911947 , 6536.09830186, 6338.64260896, 6456.70685686,
       6234.97203593, 6794.25120283, 6713.45694262, 6693.04880297,
       6610.58461646, 7085.82408339])

In [26]:
desvio_padrao = (resultado["test_score"]*-1).std()
lim_inferior = media - (2 * desvio_padrao)
lim_superior = media + (2 * desvio_padrao)

print(f"Intervalo de confiança {lim_inferior} - {lim_superior}")

Intervalo de confiança 6013.999884296945 - 7099.010114540875


In [29]:
def calcula_mse(resultado):
  media = (resultado["test_score"]*-1).mean()
  desvio_padrao = (resultado["test_score"]*-1).std()
  lim_inferior = media - (2 * desvio_padrao)
  lim_superior = media + (2 * desvio_padrao)

  print(f"Intervalo de confiança {lim_inferior} - {lim_superior}")

In [37]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
import numpy as np

SEED = 1232
np.random.seed(SEED)

pastes = KFold(n_splits=10, shuffle=True)
modelo_arvore = DecisionTreeRegressor(max_depth=3)
resultado = cross_validate(modelo_arvore, x, y, cv=pastes, scoring="neg_mean_squared_error")
calcula_mse(resultado)

Intervalo de confiança 5469.869010788644 - 5747.840733538392


In [49]:
def regressor_arvore(nivel):
  SEED = 1232
  np.random.seed(SEED)
  pastes = KFold(n_splits=10, shuffle=True)
  modelo_arvore = DecisionTreeRegressor(max_depth=nivel)
  resultado = cross_validate(modelo_arvore, x, y, cv=pastes, scoring="neg_mean_squared_error",
                             return_train_score=True)
  print(f"Treino = {(resultado['train_score']*-1).mean()} | Teste = {(resultado['test_score']*-1).mean()}")

In [50]:
for i in range(1, 21):
  regressor_arvore(i)

Treino = 7844.952652412207 | Teste = 7857.871770206351
Treino = 6527.783475408351 | Teste = 6556.886287657656
Treino = 6021.275229419074 | Teste = 6054.767567442071
Treino = 5760.834451245899 | Teste = 5815.707286597402
Treino = 5598.158955426787 | Teste = 5675.196789060743
Treino = 5469.677369145868 | Teste = 5593.304363812442
Treino = 5369.458309852723 | Teste = 5542.730565094406
Treino = 5276.16965707928 | Teste = 5543.4809609730655
Treino = 5166.713159932668 | Teste = 5608.854872163518
Treino = 5022.598049115693 | Teste = 5763.155248722027
Treino = 4833.560706021637 | Teste = 5960.052989742388
Treino = 4598.125022484774 | Teste = 6216.545956232321
Treino = 4316.3791662866815 | Teste = 6519.376217056128
Treino = 4000.6209880837496 | Teste = 6849.577463417832
Treino = 3655.8197526274853 | Teste = 7220.55475782693
Treino = 3295.832684540846 | Teste = 7634.732578617249
Treino = 2931.7212612043877 | Teste = 8068.112294631386
Treino = 2575.59534021339 | Teste = 8449.25417707589
Treino = 

Desafio 01: Pesquisar sobre intervalo de confiança.

Desafio 02: Testar outros parâmetros da árvode de decisão.

Desafio 03: Procurar outras formar de realizar os ajustes de parâmetros com o sklearn.

Desafio 04: Pesquisar o que é o problema de underfit.

Desafio 05: Plotar um gráfico do score em função do parâmetro (uma linha, ou duas com treino e teste)